In [1]:
from __future__ import print_function

# Import libraries
import numpy as np
import pandas as pd
import matplotlib
import sklearn
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties # for unicode fonts
import psycopg2
import sys
import datetime as dt
import mp_utils as mp

from collections import OrderedDict

# used to print out pretty pandas dataframes
from IPython.display import display, HTML

from sklearn.pipeline import Pipeline

# used to impute mean for data and standardize for computational stability
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# logistic regression is our favourite model ever
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV # l2 regularized regression
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier

# used to calculate AUROC/accuracy
from sklearn import metrics

# used to create confusion matrix
from sklearn.metrics import confusion_matrix

# gradient boosting - must download package https://github.com/dmlc/xgboost
import xgboost as xgb

%matplotlib inline

# below config used on pc70
sqluser = 'alistairewj'
dbname = 'mimic'
schema_name = 'mimiciii'
query_schema = 'SET search_path to public,' + schema_name + ';'

In [ ]:
USE_SQL=0

if USE_SQL:
    # Connect to local postgres version of mimic
    con = psycopg2.connect(dbname=dbname, user=sqluser)

    # exclusion criteria:
    #   - less than 15 years old
    #   - stayed in the ICU less than 4 hours
    #   - never have any chartevents data (i.e. likely administrative error)
    #   - organ donor accounts (administrative "readmissions" for patients who died in hospital)
    query = query_schema + \
    """
    select 
        *
    from dm_cohort
    where excluded = 0
    """
    co = pd.read_sql_query(query,con)

    # extract static vars into a separate dataframe
    df_static = pd.read_sql_query(query_schema + 'select * from mp_static_data', con)
    #for dtvar in ['intime','outtime','deathtime']:
    #    df_static[dtvar] = pd.to_datetime(df_static[dtvar])

    vars_static = [u'is_male', u'emergency_admission', u'age',
                   # services
                   u'service_any_noncard_surg',
                   u'service_any_card_surg',
                   u'service_cmed',
                   u'service_traum',
                   u'service_nmed',
                   # ethnicities
                   u'race_black',u'race_hispanic',u'race_asian',u'race_other',
                   # phatness
                   u'height', u'weight', u'bmi']


    # get ~5 million rows containing data from errbody
    # this takes a little bit of time to load into memory (~2 minutes)

    # %%time results
    # CPU times: user 42.8 s, sys: 1min 3s, total: 1min 46s
    # Wall time: 2min 7s

    df = pd.read_sql_query(query_schema + 'select * from mp_data', con)
    df.drop('subject_id',axis=1,inplace=True)
    df.drop('hadm_id',axis=1,inplace=True)
    df.sort_values(['icustay_id','hr'],axis=0,ascending=True,inplace=True)
    print(df.shape)

    # get death information
    df_death = pd.read_sql_query(query_schema + """
    select 
    co.subject_id, co.hadm_id, co.icustay_id
    , ceil(extract(epoch from (co.outtime - co.intime))/60.0/60.0) as dischtime_hours
    , ceil(extract(epoch from (adm.deathtime - co.intime))/60.0/60.0) as deathtime_hours
    , case when adm.deathtime is null then 0 else 1 end as death
    from dm_cohort co
    inner join admissions adm
    on co.hadm_id = adm.hadm_id
    where co.excluded = 0
    """, con)
    
    # get censoring information
    df_censor = pd.read_sql_query(query_schema + """
    select co.icustay_id, min(cs.charttime) as censortime
    , ceil(extract(epoch from min(cs.charttime-co.intime) )/60.0/60.0) as censortime_hours
    from dm_cohort co 
    inner join mp_code_status cs
    on co.icustay_id = cs.icustay_id
    where cmo+dnr+dni+dncpr+cmo_notes>0
    and co.excluded = 0
    group by co.icustay_id
    """, con)

    # get severity scores
    df_soi = pd.read_sql_query(query_schema + """
    select 
    co.icustay_id
    , case when adm.deathtime is null then 0 else 1 end as death
    , sa.saps
    , sa2.sapsii
    , aps.apsiii
    , so.sofa
    , lo.lods
    , oa.oasis
    from dm_cohort co
    inner join admissions adm
    on co.hadm_id = adm.hadm_id
    left join saps sa
    on co.icustay_id = sa.icustay_id
    left join sapsii sa2
    on co.icustay_id = sa2.icustay_id
    left join apsiii aps
    on co.icustay_id = aps.icustay_id
    left join sofa so
    on co.icustay_id = so.icustay_id
    left join lods lo
    on co.icustay_id = lo.icustay_id
    left join oasis oa
    on co.icustay_id = oa.icustay_id
    where co.excluded = 0
    """, con)
    
    # extract static vars into a separate dataframe
    df_static = pd.read_sql_query(query_schema + 'select * from mp_static_data', con)

In [2]:
USE_CSV=1

if USE_CSV:
    co = pd.read_csv('df_cohort.csv')
    df = pd.read_csv('df_data.csv')
    df_static = pd.read_csv('df_static_data.csv')
    df_censor = pd.read_csv('df_censor.csv')
    df_death = pd.read_csv('df_cohort.csv')
    df_soi = pd.read_csv('df_cohort.csv')

IOError: File df_static_data.csv does not exist

# Exclusion criteria

Each study has its own exclusion criteria (sometimes studies have multiple experiments). We define a dictionary of all exclusions with the dictionary key as the study name. Some studies have multiple experiments, so we append *a*, *b*, or *c*.

The dictionary stores a length 2 list. The first element defines the window for data extraction: it contains a dictionary of the windows and the corresponding window sizes. The second element is the exclusion criteria. Both are functions which use `co` or `df` as their input.

In [ ]:
# first we can define the different windows: there aren't that many!
df_tmp=df_death.copy().merge(df_censor, how='left', left_on='icustay_id', right_on='icustay_id').set_index('icustay_id')

# admission+12 hours
time_12hr = df_tmp.copy()
time_12hr['windowtime'] = 12
time_12hr = time_12hr['windowtime'].to_dict()

# admission+24 hours
time_24hr = df_tmp.copy()
time_24hr['windowtime'] = 24
time_24hr = time_24hr['windowtime'].to_dict()

# admission+48 hours
time_48hr = df_tmp.copy()
time_48hr['windowtime'] = 48
time_48hr = time_48hr['windowtime'].to_dict()

# admission+72 hours
time_72hr = df_tmp.copy()
time_72hr['windowtime'] = 72
time_72hr = time_72hr['windowtime'].to_dict()

# admission+96 hours
time_96hr = df_tmp.copy()
time_96hr['windowtime'] = 96
time_96hr = time_96hr['windowtime'].to_dict()

# entire stay
time_all = df_tmp.copy()
time_all = time_all['dischtime_hours'].apply(np.ceil).astype(int).to_dict()

# 12 hours before the patient died/discharged
time_predeath = df_tmp.copy()
time_predeath['windowtime'] = time_predeath['dischtime_hours']
idx = time_predeath['deathtime_hours']<time_predeath['dischtime_hours']
time_predeath.loc[idx,'windowtime'] = time_predeath.loc[idx,'deathtime_hours']
# move from discharge/death time to 12 hours beforehand
time_predeath['windowtime'] = time_predeath['windowtime']-12
time_predeath = time_predeath['windowtime'].apply(np.ceil).astype(int).to_dict()

In [ ]:
# example params used to extract patient data
# element 1: dictionary specifying end time of window for each patient
# element 2: size of window
# element 3: extra hours added to make it easier to get data on labs (and allows us to get labs pre-ICU)
# e.g. [time_24hr, 8, 24] is
#   (1) window ends at admission+24hr
#   (2) window is 8 hours long
#   (3) lab window is 8+24=32 hours long

exclusions = OrderedDict([
['caballero2015dynamically_a',  [[time_24hr, 24, 24], Random subsample]],
['caballero2015dynamically_b',  [[time_48hr, 48, 24], Random subsample]],
['caballero2015dynamically_c',  [[time_72hr, 72, 24], Random subsample]],
['calvert2016computational',    [[time_predeath, 5, 24], 18 "alcohol-use dependence related ICD-9 (291.X, 291.XX, 303.XX, 305.XX, 357.5, 425.5, 535.3X, 571.2, and 571.3)" MICU, >1 obs for all features, LOS >= 17hr, ]],
['calvert2016using',            [[time_predeath, 5, 24], 18  MICU, >1 obs for all features, LOS >= 17hr, LOS <= 500hr]],
['celi2012database_a',          [[time_72hr, 72, 24], ? implied? 18 AKI - 584.9 ]],
['celi2012database_b',          [[time_24hr, 24, 24],   SAH - 430, 852 ]],
['che2016recurrent_a',          [[time_48hr, 48, 24], ?18  ]],
['che2016recurrent_b',          [[time_48hr, 48, 24], 18  ]],
['ding2016mortality',           [[time_48hr, 48, 24], 18  ]],
['ghassemi2014unfolding_a',     [[time_24hr, 24, 24], 18  Notes must contain 100 non-stop words]],
['ghassemi2014unfolding_b',     [[time_12hr, 12, 24], 18  Notes must contain 100 non-stop words]],
['ghassemi2015multivariate',    [[time_24hr, 24, 24], 18  Notes must contain 100 non-stop words, pts have > 6 notes]],
['grnarova2016neural',          [[time_all,  24, 24], 18  "with only one hospital admission"]],
['harutyunyan2017multitask',    [[time_48hr, 48, 24], 18  "excluded any hospital admission with multiple ICU stays or transfers between different ICU units or wards"]],
['hoogendoorn2016prediction',   [[time_24hr, 24, 24], 18  1 BUN/Hct/GCS/HR/IV recording]],
['hug2009icu',                  [[time_24hr, 24, 24],  *EXCLUDE* CRF need 1 obs for HR/GCS/Hct/BUN, not NSICU/TSICU, first ICU stay, full code, not on dialysis]],
['johnson2012patient',          [[time_48hr, 48, 24], Random one third sample]],
['johnson2014data',             [[time_48hr, 48, 24], Random one third sample]],
['joshi2012prognostic',         [[time_24hr, 24, 24],   ]],
['joshi2016identifiable',       [[time_48hr, 48, 24], ]],
['lee2015customization',        [[time_24hr, 24, 24], 18  Only MICU, SICU, CCU, CSRU, no missing data]],
['lee2015personalized',         [[time_24hr, 24, 24],   "Only ICU stays with complete data"]],
['lee2017patient',              [[time_24hr, 24, 24],   Missing data, *included readmissions*]],
['lehman2012risk',              [[time_24hr, 24, 24],   Missing SAPS-I, LOS<24, first ICU stay only]],
['luo2016interpretable',        [[time_all,  24, 24], 18  No disch summary, no SAPS-II]],
['luo2016predicting',           [[time_24hr, 12, 24],   Subset of Joshi2012 with "one day length of time series data"]],
['pirracchio2015mortality',     [[time_24hr, 24, 24], >15  ]],
['ripoll2014sepsis',            [[time_24hr, 24, 24],   Missing data, only sepsis patients (sepsis not defined)]],
['wojtusiak2017c',              [[time_all,  24, 24], 65  Alive at hospital disch]]
])

In [ ]:
K=5
np.random.setseed(0.871)
# get unique subject_id (this is needed later)
sid = np.sort(np.unique(df_death['subject_id'].values))

# assign k-fold
idxK_sid = np.random.permutation(sid.shape[0])
idxK_sid = np.mod(idxK_sid,K)

# get indices which map subject_ids in sid to the X dataframe
idxMap = np.searchsorted(sid, df_death['subject_id'].values)

# use these indices to map the k-fold integers
idxK = idxK_sid[idxMap]

In [ ]:
# example of pulling data
params = exclusions['lehman2012risk']

df_data = mp.get_design_matrix(df, params[0], W=params[1], W_extra=params[2])

# load the data into a numpy array

# first, the data from static vars from df_static
X = df_data.merge(df_static.set_index('icustay_id')[vars_static], how='left', left_index=True, right_index=True)
# next, add in the outcome: death in hospital
X = X.merge(df_death.set_index('icustay_id')[['death']], left_index=True, right_index=True)

# convert to numpy data (assumes target, death, is the last column)
X = X.values
y = X[:,-1]
X = X[:,0:-1]
X_header = vars_static + [x for x in df_data.columns.values]

In [ ]:
# Rough timing info:
#     rf - 3 seconds per fold
#    xgb - 30 seconds per fold
# logreg - 4 seconds per fold
#  lasso - 8 seconds per fold
models = {'xgb': xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05),
          'lasso': LassoCV(cv=5,fit_intercept=True,normalize=True,max_iter=10000),
          'logreg': LogisticRegression(fit_intercept=True),
          'rf': RandomForestClassifier()
         }



# create k-fold indices
K = 5 # number of folds
idxK = np.random.permutation(X.shape[0])
idxK = np.mod(idxK,K)

mdl_val = dict()
results_val = dict()
pred_val = dict()
tar_val = dict()

for mdl in models:
    print('=============== {} ==============='.format(mdl))
    mdl_val[mdl] = list()
    results_val[mdl] = list() # initialize list for scores
    pred_val[mdl] = list()
    tar_val[mdl] = list()

    if mdl == 'xgb':
        # no pre-processing of data necessary for xgb
        estimator = Pipeline([(mdl, models[mdl])])

    else:
        estimator = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("scaler", StandardScaler()),
                      (mdl, models[mdl])]) 

    for k in range(K):
        # train the model using all but the kth fold
        curr_mdl = estimator.fit(X[idxK != k, :],y[idxK != k])

        # get prediction on this dataset
        if mdl == 'lasso':
            curr_prob = curr_mdl.predict(X[idxK == k, :])
        else:
            curr_prob = curr_mdl.predict_proba(X[idxK == k, :])
            curr_prob = curr_prob[:,1]
        
        pred_val[mdl].append(curr_prob)
        tar_val[mdl].append(y[idxK == k])
        
        # calculate score (AUROC)
        curr_score = metrics.roc_auc_score(y[idxK == k], curr_prob)

        # add score to list of scores
        results_val[mdl].append(curr_score)

        # save the current model
        mdl_val[mdl].append(curr_mdl)
        
        print('{} - Finished fold {} of {}. AUROC {:0.3f}.'.format(dt.datetime.now(), k+1, K, curr_score))

# Evaluate other time intervals

* 12 hour
* 24 hour
* 48 hour
* 72 hour

Require the patient to stay at least 4 hours. First define the same K-folds to be repeatedly used throughout.

## 12 hours

In [ ]:
# == 12 hours == #
W=12

# manually define the time dictionary as admission+24 hours
# since everything is relative to admission, we just fix the time to be 24 for all patients
time_dict = df_death.copy().set_index('icustay_id')
time_dict['windowtime'] = W
time_dict = time_dict['windowtime'].to_dict()
df_data = mp.get_design_matrix(df, time_dict, W=24, W_extra=24)

# get a list of icustay_id who stayed at least 12 hours
iid_min_stay = df.groupby('icustay_id')['hr'].max() >= W
iid_min_stay=iid_min_stay.index[iid_min_stay.values].values

print('Looking at the first 12 hours of the ICU stay.')
print('Reducing sample size from {} to {} ({:2.2f}%) to ensure patients stayed long enough.'.format(
        df_data.shape[0], iid_min_stay.shape[0], iid_min_stay.shape[0]*100.0 / df_data.shape[0]))
df_data = df_data.loc[iid_min_stay,:]
print('')

# load the data into a numpy array

# first, the data from static vars from df_static
X = df_data.merge(df_static.set_index('icustay_id')[vars_static], how='left', left_index=True, right_index=True)
# next, add in the outcome: death in hospital
X = X.merge(df_death.set_index('icustay_id')[['death']], left_index=True, right_index=True)

# map above K-fold indices to this dataset
X = X.merge(df_death.set_index('icustay_id')[['subject_id']], left_index=True, right_index=True)
# get indices which map subject_ids in sid to the X dataframe
idxMap = np.searchsorted(sid, X['subject_id'].values)
# use these indices to map the k-fold integers
idxK = idxK_sid[idxMap]
# drop the subject_id column
X.drop('subject_id',axis=1,inplace=True)

# convert to numpy data (assumes target, death, is the last column)
X = X.values
y = X[:,-1]
X = X[:,0:-1]
X_header = vars_static + [x for x in df_data.columns.values]

# Rough timing info:
#     rf - 3 seconds per fold
#    xgb - 30 seconds per fold
# logreg - 4 seconds per fold
#  lasso - 8 seconds per fold
models = {'xgb': xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05),
          'lasso': LassoCV(cv=5,fit_intercept=True,normalize=True,max_iter=10000),
          'logreg': LogisticRegression(fit_intercept=True),
          'rf': RandomForestClassifier()
         }

mdl_val = dict()
results_val = dict()
pred_val = dict()
tar_val = dict()

for mdl in models:
    print('=============== {} ==============='.format(mdl))
    mdl_val[mdl] = list()
    results_val[mdl] = list() # initialize list for scores
    pred_val[mdl] = list()
    tar_val[mdl] = list()

    if mdl == 'xgb':
        # no pre-processing of data necessary for xgb
        estimator = Pipeline([(mdl, models[mdl])])

    else:
        estimator = Pipeline([("imputer", Imputer(missing_values='NaN',
                                          strategy="mean",
                                          axis=0)),
                      ("scaler", StandardScaler()),
                      (mdl, models[mdl])]) 

    for k in range(K):
        # train the model using all but the kth fold
        curr_mdl = estimator.fit(X[idxK != k, :],y[idxK != k])

        # get prediction on this dataset
        if mdl == 'lasso':
            curr_prob = curr_mdl.predict(X[idxK == k, :])
        else:
            curr_prob = curr_mdl.predict_proba(X[idxK == k, :])
            curr_prob = curr_prob[:,1]
        
        pred_val[mdl].append(curr_prob)
        tar_val[mdl].append(y[idxK == k])
        
        # calculate score (AUROC)
        curr_score = metrics.roc_auc_score(y[idxK == k], curr_prob)

        # add score to list of scores
        results_val[mdl].append(curr_score)

        # save the current model
        mdl_val[mdl].append(curr_mdl)
        
        print('{} - Finished fold {} of {}. AUROC {:0.3f}.'.format(dt.datetime.now(), k+1, K, curr_score))
        
# create a pointer for above dicts with new var names
# we will likely re-use the dicts in subsequent calls for getting model perfomances
mdl_val_12 = mdl_val
results_val_12 = results_val
pred_val_12 = pred_val
tar_val_12 = tar_val